In [31]:
import pandas as pd
import numpy as np
from scipy import sparse
import random

In [32]:
inputfile = "C:\\Users\\thumm\\Documents\\machineLearning\\nlp\\data\\orcas.tsv"
data = pd.read_csv(inputfile, sep='\t', header=None)
data.columns = ['qid','query','did','doc']
data.head(3)

,qid,query,did,doc
0,9265503,github,D1265400,https://desktop.github.com/
1,9265503,github,D3438005,https://github.com/
2,6832981,youtube,D2923232,https://au.youtube.com/


In [33]:
qids = data['qid'].unique()
dids = data['did'].unique()

In [35]:
qidsmap = dict()

for i in range(qids.shape[0]):
    qidsmap[qids[i]] = i

#print(qidsmap)

didsmap = dict()
for i in range(dids.shape[0]):
    didsmap[dids[i]] = i
#print(didsmap)

In [36]:
data['qidint'] = data.apply(lambda row : qidsmap[row['qid']], axis=1)
data['didint'] = data.apply(lambda row : didsmap[row['did']], axis=1)

In [37]:
data.head(3)

,qid,query,did,doc,qidint,didint
0,9265503,github,D1265400,https://desktop.github.com/,0,0
1,9265503,github,D3438005,https://github.com/,0,1
2,6832981,youtube,D2923232,https://au.youtube.com/,1,2


In [26]:
a = data['qidint'].astype(str) + data['didint'].astype(str)

In [28]:
print(len(a.unique()))
print(len(data))

18823256
18823553


In [29]:
def sample_zero_forever(rows, cols, nonzero_or_sampled):
    while True:
        t = tuple([random.randrange(0, rows, 1), random.randrange(0, cols, 1)])
        if t not in nonzero_or_sampled:
            yield t
            nonzero_or_sampled.add(t)
    

In [38]:
dims = (data['qidint'].max(), data['didint'].max())
print(dims)

mat = sparse.csr_matrix((np.ones(data.shape[0]), (data['qidint'], data['didint'])))
#del data

rows = mat.shape[0]
cols = mat.shape[1]
nonzero_or_sampled = set(zip(*mat.nonzero()))
del mat
itr = sample_zero_forever(rows, cols, nonzero_or_sampled)
zeroedges = []

for _ in range(40000000):
    zeroedges.append(next(itr))

(10405321, 1422026)


In [47]:
q,d = zeroedges[1]
data[(data['qidint']==0) & (data['didint']==1)]

,qid,query,did,doc,qidint,didint
1,9265503,github,D3438005,https://github.com/,0,1
6457999,9265503,github,D3438005,https://github.com/,0,1


In [49]:
data.drop_duplicates(inplace=True)

In [50]:
len(data)

18823283

In [52]:
nonedges = pd.DataFrame(zeroedges, columns=['qidint', 'didint'])
print(len(nonedges))
nonedges.drop_duplicates(inplace=True)
print(len(nonedges))

#nonedgesdata = pd.merge(nonedges, data, on=['qidint', 'didint'], how='inner')

40000000
40000000


In [59]:
queries = data[['qid', 'query', 'qidint']].drop_duplicates()
docs = data[['did', 'doc', 'didint']].drop_duplicates()

In [56]:
nonedges.head(3)

,qidint,didint
0,6362512,1005356
1,5815201,958336
2,473050,1096643


In [60]:
nonedgesdata1 = nonedges.merge(queries, on='qidint', how='inner')
nonedgesdata2 = nonedgesdata1.merge(docs, on='didint', how='inner')

In [61]:
nonedgesdata2.head(3)

,qidint,didint,qid,query,did,doc
0,6362512,1005356,10375130,non qualified deferred comp plan,D3419629,https://targetjobs.co.uk/careers-advice/job-de...
1,3073276,1005356,10697799,facts about bones,D3419629,https://targetjobs.co.uk/careers-advice/job-de...
2,7180121,1005356,5767772,ptsd handout,D3419629,https://targetjobs.co.uk/careers-advice/job-de...


In [69]:
data0 = nonedgesdata2[['qid','query','did','doc']].drop_duplicates()
data0['label'] = 0

data1 = data[['qid','query','did','doc']].drop_duplicates()
data1['label'] = 1

In [70]:
res = pd.concat([data0, data1], ignore_index=True)
res.head(3)

,qid,query,did,doc,label
0,10375130,non qualified deferred comp plan,D3419629,https://targetjobs.co.uk/careers-advice/job-de...,0
1,10697799,facts about bones,D3419629,https://targetjobs.co.uk/careers-advice/job-de...,0
2,5767772,ptsd handout,D3419629,https://targetjobs.co.uk/careers-advice/job-de...,0


In [72]:
print(len(res))
res = res.sample(frac=1).reset_index(drop=True)
print(res.head(10))

import os

os.makedirs('../../data/validation', exist_ok=True)

res.to_csv('../../data/validation/approx60M.tsv', sep='\t', index=False, header=False)

58823326
        qid                     query       did  \
0  11272549  caterpillar to butterfly  D3358478   
1   2327436   dean martin thats amore  D1624588   
2   9540010                    munbai  D2549898   
3   4989404               dns 4.4.4.4  D1470300   
4   4384371         anzac day service    D88043   
5   4121834                   flirkin  D1006715   
6  12522594       art deco definition  D2173622   
7   4397811    amazon self-publishing  D2786796   
8   6428478  airplane carry on liquid  D3559716   
9  10049124   fair work notice period   D284442   

                                                 doc  label  
0  https://www.thebutterflysite.com/life-cycle.shtml      1  
1        https://www.youtube.com/watch?v=OnFlx2Lnr9Q      1  
2                http://www.india.com/travel/mumbai/      1  
3  https://www.drweil.com/vitamins-supplements-he...      0  
4            https://en.wikipedia.org/wiki/Anzac_Day      1  
5        https://en.wikipedia.org/wiki/South_Pacific     

In [91]:
temp = pd.read_csv('C:\\Users\\thumm\\Documents\\machineLearning\\nlp\\data\\validation\\splits\\approx60M_1.tsv', sep='\t', header=None)

In [92]:
temp.head()

,0,1,2,3,4
0,11272549,caterpillar to butterfly,D3358478,https://www.thebutterflysite.com/life-cycle.shtml,1
1,2327436,dean martin thats amore,D1624588,https://www.youtube.com/watch?v=OnFlx2Lnr9Q,1
2,9540010,munbai,D2549898,http://www.india.com/travel/mumbai/,1
3,4989404,dns 4.4.4.4,D1470300,https://www.drweil.com/vitamins-supplements-he...,0
4,4384371,anzac day service,D88043,https://en.wikipedia.org/wiki/Anzac_Day,1


In [93]:
temp[temp[0]==8998773]

,0,1,2,3,4
108832,8998773,http://www.w3.org/2000/svg\tD1224869\thttps://...,D619470,https://account.live.com/acsr,1


In [85]:
with open('C:\\Users\\thumm\\Documents\\machineLearning\\nlp\\data\\validation\\splits\\approx60M_1.tsv', 'r') as fp:
    fp.seek(10943398)
    for i in range(64):
        s = fp.readline().strip('\n\r')
        print(s)

8460983	madurensis calamondin	D615701	https://www.usatoday.com/story/money/columnist/brooks/2013/05/06/social-security-retirement-pension-401k/2132295/	0
8146901	eye chalazion treatment	D1192422	https://weather.com/weather/tenday/l/Ashland+AL+36251:4:US	0
7585151	san andras	D3279310	https://www.nytimes.com/2015/04/16/style/keeping-your-car-safe-from-electronic-thieves.html	0
2712820	college bowl game scores	D1229170	https://en.wikipedia.org/wiki/Time_in_Russia	0
10763058	kitchen faucets canada	D3230111	http://kmz.extensionfile.net/	0
4580505	how to get ear to pop	D440105	https://www.wikihow.com/Unpop-Your-Ears	1
7037161	phylums of animalia	D682972	https://en.wikipedia.org/wiki/NY_SAFE_Act	0
6246165	lunar eclispe	D1794903	https://support.tangohealth.com/hc/en-us/articles/220274528-IRS-HSA-Penalties	0
8428295	scott kelly astronaut divorce	D2367101	https://www.a-1truckschool.com/tuition/	0
3924331	fluid compartment	D1125306	http://antranik.org/fluid-compartments-within-the-human-body/	1
7

In [75]:
with open('../../data/validation/approx60M.tsv', 'r', encoding = 'utf-8') as fp:
    print(fp.readline())


11272549	caterpillar to butterfly	D3358478	https://www.thebutterflysite.com/life-cycle.shtml	1



In [80]:
from filesplit.split import Split
os.makedirs('../../data/validation/splits/', exist_ok=True)
split = Split('../../data/validation/approx60M.tsv', '../../data/validation/splits/')
partition_size = os.path.getsize(inputfile) / 50
split.bysize(partition_size, newline=True)

In [77]:
!pip install filesplit

  Using cached filesplit-4.0.1-py3-none-any.whl (9.2 kB)
